In [1]:
from ase.io import read
from collections import defaultdict
import numpy as np
import itertools
import geometries

In [2]:
#Read in xyz trajectory
traj = read("test-traj.xyz", format="xyz", index=':') #https://wiki.fysik.dtu.dk/ase/ase/io/io.html

#Also be able to ready in PDB / GRO / ?

In [3]:
#Trajectories will be read in as a list of ASE Atoms units
traj
#Single structs will be a single ASE Atoms unit -- differentiate between them?

[Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbols='EuN2C10O8H12OH2OH2OH2', pbc=False),
 Atoms(symbo

In [4]:
# Get central ion ID
# User can provide:
#                 Element
#                 Atom Index
# Can also be MORE THAN ONE Element/Atom Index
metal_ids, element_ids = [], []
metals = ["Eu"]
elements = ['O','N']


for i,x in enumerate(traj[0]):
    if x.symbol in metals: 
        metal_ids.append(i)
    elif x.symbol in elements: 
        element_ids.append(i)
print(metal_ids, element_ids)

[0] [1, 2, 13, 14, 15, 16, 17, 18, 19, 20, 33, 36, 39]


In [5]:
#Calculate RDF between ions and elements of interest
traj_nearby_ids_set = defaultdict(set)
rdf_frames = []
rdf_int_frames = []
rdf_max = 6.0 # Angstroms
shell_max = 4.0 # Angstroms - to help shorten first-shell geometry analyses, store only atom IDs that ever occur within 5A of metal
bin_width = 0.02
bin_range = np.arange(0, rdf_max+bin_width, bin_width)

for frame in traj:
    for metal_id in metal_ids:

        f_dists = frame.get_distances(metal_id, element_ids)

        ### To save time later, keep indices of atoms that enter first shell over traj
        near_ids_frame = [element_ids[x] for x in (f_dists<shell_max).nonzero()[0]]
        traj_nearby_ids_set[metal_id] |= set(near_ids_frame)

        bin_counts = np.bincount(np.searchsorted(bin_range, f_dists, side="left"))  # this bins the distances to bin_range 
                                                                                    # bin_counts[0] will be dists less than 0 (presumably none)
                                                                                    # bin_counts[-1] will either be the number of dists > rdf_max (if there are any) or 
                                                                                    #                                   count of dists <= a value less than rdf_max (will need padding)
        if len(bin_counts) == len(bin_range)+1: #case where there are dists > rdf_max
            bins = bin_counts[1:-1] # ignore bin_counts[0] and ignore counts of elements greater than rdf_max
            rdf_int_frames.append(np.cumsum(bins).copy())
        else:
            bins = bin_counts[1:] # ignore bin_counts[0] -- will need to pad with zeros to complete bin range
            rdf_int_frames.append(np.pad(np.cumsum(bins), (0, len(bin_range)-len(bins)), "edge").copy())
        
        dists_count = sum(bins)
        density_A = 1.0 / (4.0/3.0 * np.pi * np.power(rdf_max, 3))
        bins = 1 / (density_A*dists_count)*bins / (4.0*np.pi*np.arange(1,len(bins)+1,1)**2 * bin_width**3)

        if len(bins) < len(bin_range):
            rdf_frames.append(np.pad(bins, (0, len(bin_range)-len(bins)), "constant").copy())
        else:
            rdf_frames.append(bins.copy())

rdf = np.mean(rdf_frames, axis=0) #Calculate RDF of trajectory
rdf_int = np.mean(rdf_int_frames, axis=0) #Calculate integral of RDF of trajectory

max_rdf_FS_ind = np.argmax(rdf[0:int(shell_max/bin_width)]) #Gets the index of the RDF where the maximum RDF is (within 0 to FS screening range)
min_rdf_FS_ind = np.argmin(rdf[max_rdf_FS_ind:int(shell_max/bin_width)])+max_rdf_FS_ind #Gets the index of the RDF of the earliest minimum value after the RDF max (up to the FS screening range)

min_rdf_FS = bin_range[min_rdf_FS_ind]

coord_num = rdf_int[min_rdf_FS_ind] #Coordination number (float)

# TBD - determine handling non-close-to-integer cases (i.e. do both upper and lower CNs?)
coord_num = round(coord_num)

#with open("RDF.csv", "w") as savefile:
#    savefile.write(f'Distance,RDF,RDF_int\n')
#    for i,j in enumerate(bin_range):
#        savefile.write(f'{j:.},{rdf[i]},{rdf_int[i]}\n')



In [80]:
adf_angles = []

def calc_angle(v1_u, v2_u):
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)) * (180.0 / np.pi)

for frame in traj:
    for metal_id in metal_ids:
        candidate_FS_ids = list(traj_nearby_ids_set[metal_id])
        f_dists = frame.get_distances(metal_id, candidate_FS_ids)
        frame_FS_ids = [candidate_FS_ids[i] for i,d in enumerate(f_dists) if d<min_rdf_FS]

        if len(frame_FS_ids) != coord_num:
            print(f'Case where CN = {coord_num} and # FS atoms = {len(frame_FS_ids)}.')
        
        frame_FS_pos = frame.get_positions()[frame_FS_ids] # Get atomic positions of FS
        metal_pos = frame.get_positions()[metal_id]

        #Calculate ADF for frame and append to adf_frames
        frame_FS_pos = frame_FS_pos - metal_pos
        frame_FS_pos = [x/np.linalg.norm(x) for x in frame_FS_pos]
        for combi in itertools.combinations(frame_FS_pos, 2):
            ang = calc_angle(combi[0], combi[1])
            adf_angles.append(ang)

        #Calculate geometry RMSDs
        

adf_bin_range = np.arange(0,180+1,1.0)
adf_hist = np.histogram(adf_angles, bins=adf_bin_range)

# with open("ADF.csv", "w") as savefile:
#     savefile.write(f' Angle,ADF\n')
#     for i,j in enumerate(adf_hist[0]):
#         savefile.write(f'{adf_hist[1][i]},{j}\n') #Note that saved values will be the lower-bound (i.e. 0 = 0-1, 1 = 1-2, etc.)


In [94]:
geometries.idealGeometries(coord_num, bin_range[max_rdf_FS_ind])

[['capped-square-antiprismatic',
  array([[ 2.0143114 ,  1.19771672,  0.        ],
         [-2.0143114 ,  1.19771672,  0.        ],
         [ 0.        ,  1.19771672,  2.0143114 ],
         [ 0.        ,  1.19771672, -2.0143114 ],
         [ 1.42433325, -1.19771672,  1.42433325],
         [-1.42433325, -1.19771672,  1.42433325],
         [ 1.42433325, -1.19771672, -1.42433325],
         [-1.42433325, -1.19771672, -1.42433325],
         [ 0.        ,  3.21202813,  0.        ]])],
 ['capped-square',
  array([[ 1.90872442,  1.34967198,  0.        ],
         [-1.90872442,  1.34967198,  0.        ],
         [ 0.        ,  1.34967198,  1.90872442],
         [ 0.        ,  1.34967198, -1.90872442],
         [ 1.90872442, -1.34967198,  0.        ],
         [-1.90872442, -1.34967198,  0.        ],
         [ 0.        , -1.34967198,  1.90872442],
         [ 0.        , -1.34967198, -1.90872442],
         [ 0.        ,  3.25839641,  0.        ]])],
 ['tricapped-trigonal-prismatic',
  array(